In [ ]:
# Mount Google Drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
""" Installation of library are mentioned here """
!pip install happytransformer
from IPython.display import clear_output
clear_output()

In [ ]:
""" Imports are mentioned here """

import csv
from datasets import load_dataset
from happytransformer import TTSettings
from happytransformer import TTTrainArgs
from happytransformer import HappyTextToText

In [ ]:
import pandas as pd
import zipfile

from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
from torch import optim
import torch.nn.functional as F
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from sklearn.model_selection import train_test_split
from nltk.translate import chrf_score,bleu_score
import re

import os
from typing import Iterable, List
import math
from nltk.translate import chrf_score,bleu_score
import pathlib as pl
import random

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import random
import csv
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
""" Model """
# T5 comes in several different sizes, and we'll use the base model,
# which has 220 million parameters.
# T5 is a text-to-text model, meaning given text, it generated a standalone
#  piece of text based on the input.

happy_tt = HappyTextToText("T5", "t5-base")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your original CSV file
filepath = '/content/drive/MyDrive/AML/small_dataframe.csv'
df_original = pd.read_csv(filepath)

# Set columns of the DataFrame
df_original.columns = ['input', 'target']

df_original

,input,target
0,The effect of widespread dud targets two face ...,"1. The effect of ""widespread dud"" targets two ..."
1,tax on sales of stores for non residents are s...,Capital Gains tax on the sale of properties fo...
2,Much many brands and sellers still in the market.,Many brands and sellers still in the market.
3,this is is the latest Maintenance release of S...,This is is the latest maintenance release of S...
4,"Fairy Or Not, I'm the Godmother: no just look,...","Fairy Or Not, I'm the Godmother: Not just a lo..."
...,...,...
9995,Problematic however: the Panorex is a 2 diment...,Problematic however: the Panorex is a 2 dimens...
9996,If that’s a multitude of yes then1 you’ve got ...,If that’s a multitude of yeses then you’ve fou...
9997,"Following the acquisition, he was appointed as...","Following the acquisition, he was appointed Co..."
9998,"So, don’t forget that you heard it here first:...","So, don’t forget that you heard it here first:..."


In [ ]:
# Split the data into training and evaluation sets
train_df, eval_df = train_test_split(df_original, test_size=0.2, random_state=42)

# Save the training and evaluation sets to new CSV files
train_csv_path = '/content/drive/MyDrive/AML/Data_Fine_Tuning/train.csv'
eval_csv_path =  '/content/drive/MyDrive/AML/Data_Fine_Tuning/eval.csv'

train_df.to_csv(train_csv_path, index=False)
eval_df.to_csv(eval_csv_path, index=False)

# Convert the DataFrame to a 2D list
test_pairs = eval_df.values.tolist()

# Display information about the new datasets
print("Train dataset shape:", train_df.shape)
print("Eval dataset shape:", eval_df.shape)



Train dataset shape: (8000, 2)
Eval dataset shape: (2000, 2)


In [ ]:
before_result = happy_tt.eval(eval_csv_path)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
# # The result is a dataclass object with a single variable called loss,
#  which we can isolate as shown below.
print("Before loss:", before_result.loss)

Before loss: 1.583791971206665


In [ ]:
# We can do so by calling happy_tt's train() method.
# For simplicity, we'll use the default parameters other
# than the batch size which we'll increase to 8.
args = TTTrainArgs(batch_size=8)
happy_tt.train(train_csv_path, args=args)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/7200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
1,1.207600,1.473068
90,1.005700,0.795893
180,0.769800,0.713405
270,0.749400,0.669020
360,0.724600,0.653179
450,0.682100,0.647447
540,0.675100,0.644514
630,0.681300,0.636459
720,0.642200,0.635680
810,0.645900,0.632346


In [ ]:
# Like before, let's determine the model's loss.
after_loss = happy_tt.eval(eval_csv_path)

print("After loss: ", after_loss.loss)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

After loss:  0.6577830910682678


In [ ]:
# Let's now use the model to correct the grammar of examples we'll provide it.
# To accomplish this, we'll use happy_tt's generate_text() method.

beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=20)

In [ ]:
""" Example1: """
example_1 = "grammar: This sentences, has bads grammar and spelling!"
result_1 = happy_tt.generate_text(example_1, args=beam_settings)
print(result_1.text)

grammar: This sentences, has bad grammar and spelling!


In [ ]:
""" Example2: """

example_2 = "grammar: I am enjoys, writtings articles ons AI and I also enjoyed write articling on AI."

result_2 = happy_tt.generate_text(example_2, args=beam_settings)
print(result_2.text)

grammar: I enjoy writing articles on AI and I also enjoy writing articles on AI.


In [ ]:
# We can evaluate random sentences from the training set and print out the input,
#  target, and output to make some subjective quality judgements:
# we also find the chrf_score here

def evaluateRandomly(test_pairs, n = 30):
    score = 0
    for i in range(n):
        pair = random.choice(test_pairs)
        src, trg = pair
        print('>', src)
        print('=', trg)
        prediction = happy_tt.generate_text(src, args=beam_settings)
        score = score + chrf_score.sentence_chrf(prediction.text, trg)
        print('<', prediction.text)
        print('')
    return score

In [ ]:
chrf_score = evaluateRandomly(test_pairs)

In [ ]:
# chrf score which is similarity matrixs
print(chrf_score)

44.28760803509916
